# Testing StarDist network for instance segmentation

In [2]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib
matplotlib.rcParams["image.interpolation"] = 'none'
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
from tqdm import tqdm
import skimage as sk
from tifffile import imread
from csbdeep.utils import Path, normalize
import napari
import os
from stardist import fill_label_holes, random_label_cmap, calculate_extents, gputools_available
from stardist.matching import matching, matching_dataset
from stardist.models import Config2D, StarDist2D, StarDistData2D

np.random.seed(42)
lbl_cmap = random_label_cmap()

Use the following 2 cells to convert pngs of masks exported from QuPath to Tiffs, if needed

In [ ]:
PNGs = sorted(glob('E:/Grainger_Lab/Amber/OIC-74_Zebrafish_RBC_Classification/tiles/**/instTiles/*.png',recursive=True))

In [ ]:
#Converted all pngs to tiffs
for y in PNGs:
    path = os.path.dirname(y)
    name = os.path.basename(y)
    img = sk.io.imread(y)
    sk.io.imsave(os.path.join(path,name[:-4]+'.tif'),img,check_contrast=False)

Had to copy all images and masks as tiffs to a single pair of folders:

In [53]:
for i in range(len(X)):
    sk.io.imsave(os.path.join('E:/Grainger_Lab/Amber/OIC-74_Zebrafish_RBC_Classification/tiles/All_Imgs/Imgs','Img_0'+str(i)+'.tif'),X[i],check_contrast=False)
    sk.io.imsave(os.path.join('E:/Grainger_Lab/Amber/OIC-74_Zebrafish_RBC_Classification/tiles/All_Imgs/Masks','Img_0'+str(i)+'.tif'),Y[i],check_contrast=False)
print('Done!')

Done!


Read in images

In [78]:
Xfiles = sorted(glob('E:/Grainger_Lab/Amber/OIC-74_Zebrafish_RBC_Classification/tiles/All_Imgs/Imgs/*.tif'))
Yfiles = sorted(glob('E:/Grainger_Lab/Amber/OIC-74_Zebrafish_RBC_Classification/tiles/All_Imgs/Masks/*.tif'))
#assert all(Path(x).name==Path(y).name for x,y in zip(X,Y)) #added indexing here because the file type is different between the images and the masks, using indexing to match the name of the file without file type

In [85]:
X = list(map(imread,Xfiles))
Y = list(map(imread,Yfiles))

In [86]:
X = X[0:len(X):100]

In [87]:
Y = Y[0:len(Y):100]

In [88]:
len(Y)

121

In [89]:
n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]

In [90]:
# Check number of channels
print(n_channel)

3


Normalize the images, fill possible holes in labels then split into train and validate groups

In [91]:
axis_norm = (0,1)   # normalize channels independently
# axis_norm = (0,1,2) # normalize channels jointly
if n_channel > 1:
    print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))
    sys.stdout.flush()

X = [normalize(x,1,99.8,axis=axis_norm) for x in tqdm(X)]
Y = [fill_label_holes(y) for y in tqdm(Y)]

Normalizing image channels independently.


100%|████████████████████████████████████████████████████████████████████████████████| 121/121 [00:02<00:00, 41.94it/s]


In [92]:
assert len(X) > 1, "not enough training data"
rng = np.random.RandomState(42)
ind = rng.permutation(len(X))
n_val = max(1, int(round(0.15 * len(ind))))
ind_train, ind_val = ind[:-n_val], ind[-n_val:]
X_val, Y_val = [X[i] for i in ind_val]  , [Y[i] for i in ind_val]
X_trn, Y_trn = [X[i] for i in ind_train], [Y[i] for i in ind_train] 
print('number of images: %3d' % len(X))
print('- training:       %3d' % len(X_trn))
print('- validation:     %3d' % len(X_val))

number of images: 121
- training:       103
- validation:      18


Set up hyperparameters for StarDist Model

In [93]:
# 32 is a good default choice (see 1_data.ipynb)
n_rays = 32

# Use OpenCL-based computations for data generator during training (requires 'gputools')
use_gpu = True and gputools_available()

# Predict on subsampled grid for increased efficiency and larger field of view
grid = (4,4)

conf = Config2D (
    n_rays       = n_rays,
    grid         = grid,
    use_gpu      = use_gpu,
    n_channel_in = n_channel,
    train_patch_size = (512,512),
    train_steps_per_epoch = 100,
    train_epochs = 400,
)
#print(conf)
#vars(conf)

In [94]:
model = StarDist2D(conf, name='BloodCells', basedir='models')

Using default values: prob_thresh=0.5, nms_thresh=0.4.


Make sure that the field of view for the network is larger than the objects

In [95]:
median_size = calculate_extents(list(Y), np.median)
fov = np.array(model._axes_tile_overlap('YX'))
print(f"median object size:      {median_size}")
print(f"network field of view :  {fov}")
# if any(median_size > fov):
#     print("WARNING: median object size larger than field of view of the neural network.")

functional.py (238): The structure of `inputs` doesn't match the expected structure.
Expected: ['input']
Received: inputs=Tensor(shape=(1, 512, 512, 3))


median object size:      [93.5 97. ]
network field of view :  [188 188]


Define the augementations to use

In [96]:
def random_fliprot(img, mask): 
    assert img.ndim >= mask.ndim
    axes = tuple(range(mask.ndim))
    perm = tuple(np.random.permutation(axes))
    img = img.transpose(perm + tuple(range(mask.ndim, img.ndim))) 
    mask = mask.transpose(perm) 
    for ax in axes: 
        if np.random.rand() > 0.5:
            img = np.flip(img, axis=ax)
            mask = np.flip(mask, axis=ax)
    return img, mask 

def random_intensity_change(img):
    img = img*np.random.uniform(0.6,2) + np.random.uniform(-0.2,0.2)
    return img


def augmenter(x, y):
    """Augmentation of a single input/label image pair.
    x is an input image
    y is the corresponding ground-truth label image
    """
    x, y = random_fliprot(x, y)
    x = random_intensity_change(x)
    # add some gaussian noise
    sig = 0.02*np.random.uniform(0,1)
    x = x + sig*np.random.normal(0,1,x.shape)
    return x, y

Train the model

Use `tensorboard --logdir=.` in the command line in the same parent directory as the models (with StarDist env active) to watch live read out of training (could also put in the directory of the parent folder instead of cd to parent directory)

In [97]:
model.train(X_trn, Y_trn, validation_data=(X_val,Y_val), augmenter=augmenter)

Epoch 1/400
100/100 ━━━━━━━━━━━━━━━━━━━━ 70s 634ms/step - dist_dist_iou_metric: 0.0272 - dist_loss: 38.0525 - dist_relevant_mae: 38.0520 - dist_relevant_mse: 1992.6140 - loss: 8.0935 - prob_kld: 0.3139 - prob_loss: 0.4830 - val_dist_dist_iou_metric: 0.2055 - val_dist_loss: 23.3044 - val_dist_relevant_mae: 23.0171 - val_dist_relevant_mse: 863.8041 - val_loss: 4.9043 - val_prob_kld: 0.1250 - val_prob_loss: 0.3080 - learning_rate: 3.0000e-04
Epoch 2/400
100/100 ━━━━━━━━━━━━━━━━━━━━ 60s 600ms/step - dist_dist_iou_metric: 0.3563 - dist_loss: 19.5536 - dist_relevant_mae: 19.5497 - dist_relevant_mse: 636.8657 - loss: 4.2674 - prob_kld: 0.1894 - prob_loss: 0.3566 - val_dist_dist_iou_metric: 0.3228 - val_dist_loss: 18.6850 - val_dist_relevant_mae: 18.5224 - val_dist_relevant_mse: 609.8640 - val_loss: 4.1364 - val_prob_kld: 0.2568 - val_prob_loss: 0.4399 - learning_rate: 3.0000e-04
Epoch 3/400
100/100 ━━━━━━━━━━━━━━━━━━━━ 59s 592ms/step - dist_dist_iou_metric: 0.3935 - dist_loss: 17.3621 - dist_

In [98]:
model.optimize_thresholds(X_val, Y_val)

functional.py (238): The structure of `inputs` doesn't match the expected structure.
Expected: ['input']
Received: inputs=Tensor(shape=(1, 1024, 1024, 3))
NMS threshold = 0.5:  80%|████████████████████████████████████         | 16/20 [00:35<00:08,  2.22s/it, 0.466 -> 0.941]


Using optimized values: prob_thresh=0.464147, nms_thresh=0.3.
Saving to 'thresholds.json'.


{'prob': 0.4641472399234772, 'nms': 0.3}

In [99]:
Y_val_pred = [model.predict_instances(x, n_tiles=model._guess_n_tiles(x), show_tile_progress=False)[0]
              for x in tqdm(X_val)]

100%|████████████████████████████████████████████████████████████████████████████| 18/18 [00:08<00:00,  2.03it/s]


In [106]:
import random
nums = range(len(Y_val_pred))
i = random.randint(min(nums),max(nums))

In [107]:
viewer = napari.view_image(X_val[i],name='img')
viewer.add_image(Y_val[i],name='GT')
viewer.add_image(Y_val_pred[i],name='Pred')

<Image layer 'Pred' at 0x1db557025a0>

In [114]:
X_test = list(map(imread,Xfiles))

In [115]:
X_test = X_test[7:len(Xfiles):100]

In [116]:
axis_norm = (0,1)   # normalize channels independently
# axis_norm = (0,1,2) # normalize channels jointly
if n_channel > 1:
    print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))
    sys.stdout.flush()

X_test = [normalize(x,1,99.8,axis=axis_norm) for x in tqdm(X_test)]

Normalizing image channels independently.


100%|██████████████████████████████████████████████████████████████████████████| 121/121 [00:10<00:00, 11.67it/s]


In [117]:
Y_test = [model.predict_instances(x, n_tiles=model._guess_n_tiles(x), show_tile_progress=False)[0]
              for x in tqdm(X_test)]

100%|██████████████████████████████████████████████████████████████████████████| 121/121 [00:57<00:00,  2.10it/s]


In [130]:
import random
nums = range(len(Y_test))
i = random.randint(min(nums),max(nums))

In [131]:
viewer = napari.view_image(X_test[i],name='img')
#viewer.add_image(Y_val[i],name='GT')
viewer.add_image(Y_test[i],name='Pred')

<Image layer 'Pred' at 0x1db43813200>